In [224]:
import json
import random
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import glob
import datetime
import operator


def load_metadata (datadir):
    filenames = glob.glob(datadir + '/*', recursive=False)
    filenames = [file for file in filenames if not file.endswith("png")]
    
    meta = list()
    for file in filenames:
        f = open(file)
        
        m = json.loads(f.read().replace("'", '"'))
        m["gps"] = get_lat_long(0)
        m["timestamp"] = datetime.datetime.strptime(m["timestamp"], '%Y-%m-%d %H:%M:%S.%f')
        m["filename"] = file.split("\\")[-1]
        meta.append(m)
        
        f.close()
        
    return meta

# Latitude, longitude
def get_lat_long (data):
    return random.uniform(33.824, 36.35), random.uniform(-84, -75)

def get_timestamp (data):
    return random.randint(1, 366)

metadata = load_metadata("../../data/metadata")

metadata =  metadata[:1000]
metadata = sorted(metadata, key=operator.itemgetter("timestamp"))

for m in metadata:
     m['timestamp'] = m["timestamp"].date().strftime('%m/%d/%Y')

gps = [m["gps"] for m in metadata]
dates = [m["timestamp"] for m in metadata]

seen = set()
dates = [d for d in dates if d not in seen and not seen.add(d)]

print(dates)

['06/01/2015', '06/02/2015', '06/03/2015', '06/05/2015', '06/06/2015', '06/07/2015', '06/08/2015', '06/09/2015', '06/12/2015', '06/13/2015', '06/14/2015', '06/15/2015', '06/17/2015', '06/18/2015', '06/19/2015', '06/20/2015', '06/21/2015', '06/22/2015', '06/23/2015', '06/24/2015', '06/25/2015', '06/26/2015', '06/27/2015', '06/30/2015', '07/01/2015', '07/03/2015', '07/05/2015', '07/06/2015', '07/08/2015', '07/09/2015', '07/10/2015', '07/11/2015', '07/12/2015', '07/14/2015', '07/15/2015', '07/16/2015', '07/17/2015', '07/18/2015', '06/11/2756']


In [230]:
from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

figure = {
    'data': [],
    'layout': {},
    'frames': []
}

figure['layout'] = {
    'hovermode': 'closest',
    'geo': {
        'scope': 'usa',
        'projection': dict(type = 'albers usa'),
        'showland': True,
        'landcolor': "rgb(250, 250, 250)",
        'subunitcolor': "rgb(217, 217, 217)",
        'countrycolor': "rgb(217, 217, 217)",
        'countrywidth': 0.5,
        'subunitwidth': 0.5    
    },
    'updatemenus': [{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }]
}

figure['data'] = [
    go.Scattergeo(
        locationmode= 'USA-states',
        lon= [m['gps'][1] for m in metadata if m['timestamp'] == dates[0]],
        lat= [m['gps'][0] for m in metadata if m['timestamp'] == dates[0]],
        text = [m['filename'] for m in metadata if m['timestamp'] == dates[0]],
        mode= 'markers'
    )]
    
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 18},
        'prefix': 'Date: ',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

for date in dates:

    frame = {
        'name': str(date),
        'data': 
        [dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = [call["gps"][1]], 
            lat = [call["gps"][0]], 
            mode = 'markers',
            text = call['filename']
        )  for call in metadata if call["timestamp"] == date]
    }

    figure['frames'].append(frame)
    
    slider_step = {
        'args': [
            [date],
            {
                'frame': {
                    'duration': 300, 
                    'redraw': False
                },
                'mode': 'immediate',
                'transition': {
                    'duration': 300
                }
            }
         ],
         'label': date,
         'method': 'animate'
    }
    
    sliders_dict['steps'].append(slider_step)
    
figure['layout']['sliders'] = [sliders_dict]
iplot(figure, filename="map_test")

In [202]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
import numpy as np

init_notebook_mode(connected=True)

t=np.linspace(-1,1,100)
x=t+t**2
y=t-t**2
xm=np.min(x)-1.5
xM=np.max(x)+1.5
ym=np.min(y)-1.5
yM=np.max(y)+1.5
N=50
s=np.linspace(-1,1,N)
xx=s+s**2
yy=s-s**2

data=[dict(x=x, y=y, 
           mode='lines', 
           line=dict(width=2, color='blue')
          ),
      dict(x=x, y=y, 
           mode='lines', 
           line=dict(width=2, color='blue')
          )
    ]

layout=dict(xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
            yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
            title='Kinematic Generation of a Planar Curve', hovermode='closest',
            updatemenus= [{'type': 'buttons',
                           'buttons': [{'label': 'Play',
                                        'method': 'animate',
                                        'args': [None]}]}])

frames=[dict(data=[dict(x=[xx[k]], 
                        y=[yy[k]], 
                        mode='markers', 
                        marker=dict(color='red', size=10)
                        )
                  ]) for k in range(N)]    

print (frames)

figure1=dict(data=data, layout=layout, frames=frames)          
iplot(figure1)

#py.icreate_animations(figure, filename='animation_test'+str(time.time()))

[{'data': [{'x': [0.0], 'y': [-2.0], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.03915035401915867], 'y': [-1.8792169929196167], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.07496876301541022], 'y': [-1.7617659308621407], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.10745522698875476], 'y': [-1.6476468138275717], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.13660974593919195], 'y': [-1.5368596418159102], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.16243231986672224], 'y': [-1.4294044148271552], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.1849229487713453], 'y': [-1.3252811328613077], 'mode': 'markers', 'marker': {'color': 'red', 'size': 10}}]}, {'data': [{'x': [-0.20408163265306123], 'y': [-1.2244897959183674], 'mode': 'markers', 'marker': {'color': 'red', 'size': 

In [70]:
figure['data'] = {
    'type': 'scattergeo',
    'locationmode': 'USA-states',
    'lon': [m['gps'][1] for m in metadata if m['timestamp'].date().strftime('%m/%d/%Y') == timestamp[0]],
    'lat': [m['gps'][0] for m in metadata if m['timestamp'].date().strftime('%m/%d/%Y') == timestamp[0]],
    'mode': 'markers'
}